In [1]:
!npx hardhat compile

Nothing to compile


In [1]:
import utils
utils.w3 = utils.setup_web3_provider('alchemy', 'kovan')
utils._CONTRACTS_PATH = '../artifacts/contracts'
from utils import *

In [2]:
w3.eth.get_block_number()

30460744

In [75]:
# for web3 basics
def send_tx(acct, func, args, kwargs):
    tx = func(*args).buildTransaction({
        'from': acct.address,
        'nonce': w3.eth.getTransactionCount(acct.address),
        **kwargs,
    })
    signed_tx = acct.sign_transaction(tx)
    tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    assert tx_receipt['status'] == 1
    return tx_receipt

def _load_artifact(name):
    path = os.path.join('../artifacts/contracts', f'{name}.sol')
    path = os.path.join(path, f'{name}.json')
    with open(path, 'r') as f:
        obj = json.load(f)
    return obj['abi'], obj['bytecode']

def deploy_contract(acct, name, args, kwargs):
    abi, bytecode = _load_artifact(name)
    instance = w3.eth.contract(abi=abi, bytecode=bytecode)
    tx_receipt = send_tx(
        acct,
        instance.constructor,
        args,
        kwargs,
    )
    return Cont(tx_receipt['contractAddress'], abi)

In [36]:
# default tx settings
tx_args = {
    'gasPrice': int(1.5 * 10 ** 9),
}

Initialize the contracts

In [4]:
# EOA
owner_pk = '329351ff4bcaa0baf02f0d2b273de9f92905fd4a9058fa0ec4417d4330a25578'
alice_pk = '5d021183eadbe1fdafff0cbdd78e4d314fc2ffb9624155a4235ec5c8a5f3c505'
bob_pk = 'ec90eed1fd0546c74af07d59a186bee613da20984ceb1568644392b58842d8dd'
cathy_pk = '21a61c933aa1813d9e4d69c89e80a63da37dbeaa752d4d2f5818ec174f71ed4f'
owner = w3.eth.account.privateKeyToAccount(owner_pk)
alice = w3.eth.account.privateKeyToAccount(alice_pk)
bob = w3.eth.account.privateKeyToAccount(bob_pk)
cathy = w3.eth.account.privateKeyToAccount(cathy_pk)

# tokens
weth = Cont('0xd0A1E359811322d97991E03f863a0C30C2cF029C', 'weth')
dai = Cont('0xFf795577d9AC8bD7D90Ee22b6C1703490b6512FD', 'erc20')
usdt = Cont('0x13512979ADE267AB5100878E2e0f485B568328a4', 'erc20')
wbtc = Cont('0xD1B98B6607330172f1D991521145A22BCe793277', 'erc20')
link = Cont('0xAD5ce863aE3E4E9394Ab43d4ba0D80f419F61789', 'erc20')

In [18]:
# start sending txs
feeder = owner
def set_token_price(token, price):
    send_tx(feeder, oracle.functions.setTokenPrice, [token, price], tx_args)
    
# setup oracle
input('> deploy oracle ')
oracle = deploy_contract(owner, 'Oracle', [owner.address], tx_args)
abi, bytecode = _load_artifact('Oracle')
oracle = Cont('0x5f4b75950f8fF8C68Bdb695757eaBEcA24D18E29', abi)

# setup price
input('> set price dai ')
set_token_price(dai.address, e2w(0.5))
# set_token_price(usdt.address, e2w(1)) # only for MarketV2
input('> set price wbtc ')
set_token_price(wbtc.address, e2w(10))
input('> set price link ')
set_token_price(link.address, e2w(3))

> set price dai 
> set price wbtc 
> set price link 


In [20]:
# setup challenge
input('> deploy market ')
pc = deploy_contract(owner, 'Market', [oracle.address], tx_args)
# send_tx(owner, pc.functions.setValidDuration, [9999999999], tx_args) # only for MarketV2
input('> set liq thre ')
send_tx(owner, pc.functions.setLiquidationThreshold, [8000], tx_args)
input('> set liq bouns ')
send_tx(owner, pc.functions.setLiquidationBonus, [1000], tx_args)

> deploy market 
> set liq thre 
> set liq bouns 


AttributeDict({'transactionHash': HexBytes('0xa26feff6d489d626c70db4e4a6fea74c714faad0b0e89e87dbeea6e830308432'),
 'blockHash': HexBytes('0x0daafe99241f15ccee34381016db38872ad3487aad73dbf628c9fd3f20cfb5c8'),
 'blockNumber': 30460928,
 'contractAddress': None,
 'cumulativeGasUsed': 289627,
 'effectiveGasPrice': 2000000000,
 'from': '0x85BFba62f1b0bb5ad63Dc15611B96A3457095390',
 'gasUsed': 46068,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': '0x16537776395108789FE5cC5

In [22]:
# setup allowed tokens
allowed_lock_tokens = [zero_addr, weth.address, wbtc.address, link.address]
allowed_borrow_tokens = [dai.address, usdt.address]
for token in allowed_lock_tokens:
    input('> set allowed lock token ')
    send_tx(owner, pc.functions.setAllowedLockToken, [token], tx_args)
for token in allowed_borrow_tokens:
    input('> set allowed borrow token ')
    send_tx(owner, pc.functions.setAllowedBorrowToken, [token], tx_args)

> set allowed lock token 
> set allowed lock token 
> set allowed lock token 
> set allowed lock token 
> set allowed borrow token 
> set allowed borrow token 


In [23]:
# setup token decimals
for token in [wbtc, link, dai, usdt]:
    decimals = token.functions.decimals().call()
    input('> set token decimals ')
    send_tx(owner, pc.functions.setTokenDecimals, [token.address, decimals], tx_args)
# input('> set eth decimals ')
# send_tx(owner, pc.functions.setTokenDecimals, [zero_addr, 18], tx_args)

> set token decimals 
> set token decimals 
> set token decimals 
> set token decimals 


In [24]:
# owner transfer all dai and usdt to pc
input('> transfer dai to market ')
send_tx(owner, dai.functions.transfer, [pc.address, Bal(dai, owner.address)], tx_args)
input('> transfer usdt to market ')
send_tx(owner, usdt.functions.transfer, [pc.address, Bal(usdt, owner.address)], tx_args)

> transfer dai to market 
> transfer usdt to market 


AttributeDict({'blockHash': HexBytes('0x4f8fc6fc51b6394cb86e61072166c7d66a4cb9697efa0c550e55a8f736dc9e13'),
 'blockNumber': 30461429,
 'contractAddress': None,
 'cumulativeGasUsed': 430410,
 'effectiveGasPrice': 1900000000,
 'from': '0x85BFba62f1b0bb5ad63Dc15611B96A3457095390',
 'gasUsed': 46745,
 'logs': [AttributeDict({'address': '0x13512979ADE267AB5100878E2e0f485B568328a4',
   'blockHash': HexBytes('0x4f8fc6fc51b6394cb86e61072166c7d66a4cb9697efa0c550e55a8f736dc9e13'),
   'blockNumber': 30461429,
   'data': '0x00000000000000000000000000000000000000000000000000000002540be400',
   'logIndex': 8,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x00000000000000000000000085bfba62f1b0bb5ad63dc15611b96a3457095390'),
    HexBytes('0x00000000000000000000000016537776395108789fe5cc5420545cab210a7d30')],
   'transactionHash': HexBytes('0xed27153614a76f13be46d41a187de84ae45200496c805edc6a00e4996c2596d0'),
   'transac

Start interacting with the contracts

In [38]:
# helper functions
def create_position(acct, lock_token, borrow_token):
    tx_receipt = send_tx(acct, pc.functions.createPosition, [lock_token, borrow_token], tx_args)
    return pc.events.CreatePosition().processReceipt(tx_receipt)[0]['args']['pid']

def lock(acct, pid, amount):
    is_eth = pc.functions.positions(acct.address, pid).call()[1] == zero_addr
    value = amount if is_eth else 0
    return send_tx(acct, pc.functions.lock, [pid, amount], {'value': value, **tx_args})

def unlock(acct, pid, amount):
    return send_tx(acct, pc.functions.unlock, [pid, amount], tx_args)

def borrow(acct, pid, amount):
    return send_tx(acct, pc.functions.borrow, [pid, amount], tx_args)

def repay(acct, pid, amount):
    return send_tx(acct, pc.functions.repay, [pid, amount], tx_args)

def liquidate(liquidator, user, pid, amount):
    return send_tx(liquidator, pc.functions.liquidate, [user, pid, amount], tx_args)

def get_position(acct, pid):
    return pc.functions.positions(acct.address, pid).call()

def get_health_factor(acct, pid):
    return w2e(pc.functions.getHealthFactor(acct.address, pid).call())

In [79]:
import time
from numpy import random

def sleep(secs):
    time.sleep(abs(random.normal() * secs))

In [ ]:
input('0> ')
create_position(alice, link.address, dai.address)
input('1> ')
send_tx(alice, link.functions.approve, [pc.address, uint_max], tx_args)
input('2> ')
lock(alice, 0, e2w(0.01))
input('3> ')
borrow(alice, 0, e2w(0.001))
input('4> ')
set_token_price(link.address, e2w(2.8))
input('5> ')
borrow(alice, 0, e2w(0.01))
input('6> ')
set_token_price(link.address, e2w(2.7))
input('7> ')
borrow(alice, 0, e2w(0.012))

In [69]:
sleep()#input('8> ')
create_position(bob, link.address, dai.address)
sleep()#input('9> ')
set_token_price(link.address, e2w(2.54))
sleep()#input('10> ')
set_token_price(link.address, e2w(2.6))

In [76]:
input('11> ')
borrow(alice, 0, e2w(0.01))
input('12> ')
send_tx(bob, link.functions.approve, [pc.address, uint_max], tx_args)
input('13> ')
lock(bob, 0, e2w(50))
input('14> ')
set_token_price(link.address, e2w(3.1))
input('15> ')
borrow(bob, 0, e2w(10))
input('16> ')
borrow(bob, 0, e2w(226))
input('17> ')
set_token_price(link.address, e2w(3))
input('18> ')
send_tx(alice, dai.functions.approve, [pc.address, uint_max], tx_args)
input('19> ')
repay(alice, 0, e2w(0.012))
input('20> ')
set_token_price(link.address, e2w(2.77))

12> 
13> 
14> 
15> 
16> 
17> 
18> 
19> 
20> 


In [77]:
input('21> ')
send_tx(bob, dai.functions.approve, [pc.address, uint_max], tx_args)
input('22> ')
repay(bob, 0, e2w(31))
input('23> ')
set_token_price(link.address, e2w(2.501))
input('24> ')
send_tx(cathy, dai.functions.approve, [pc.address, uint_max], tx_args)
input('25> ')
liquidate(cathy, bob.address, 0, e2w(49))
input('26> ')
set_token_price(link.address, e2w(2.4))
input('27> ')
liquidate(cathy, bob.address, 0, e2w(37.88))
input('28> ')
liquidate(cathy, bob.address, 0, e2w(21.3))
input('29> ')
repay(bob, 0, e2w(40))
input('30> ')
create_position(bob, wbtc.address, dai.address)

21> 
22> 
23> 
24> 
25> 
26> 
27> 
28> 
29> 
30> 


1

In [78]:
print('31> '), sleep()
set_token_price(link.address, e2w(2.2))
print('32> '), sleep()
create_position(alice, wbtc.address, dai.address)
print('33> '), sleep()
set_token_price(wbtc.address, e2w(12))
print('34> '), sleep()
send_tx(alice, wbtc.functions.approve, [pc.address, uint_max], tx_args)
print('35> '), sleep()
lock(alice, 1, 5 * 10**7)
print('36> '), sleep()
borrow(alice, 1, e2w(3.98))
print('37> '), sleep()
set_token_price(wbtc.address, e2w(11.5))
print('38> '), sleep()
set_token_price(link.address, e2w(2.144))
print('39> '), sleep()
unlock(alice, 1, 1 * 10**7)
print('40> '), sleep()
lock(bob, 0, e2w(50))

31> 
32> 
33> 
34> 
35> 
36> 
37> 
38> 
39> 
40> 


AttributeDict({'blockHash': HexBytes('0x986940a354b39f834c8ee299ce2dbe76612436f0b5e5bd3bfa46e3245cc7a7a8'),
 'blockNumber': 30462235,
 'contractAddress': None,
 'cumulativeGasUsed': 817866,
 'effectiveGasPrice': 1500000000,
 'from': '0xB09c603eA024b4435D74db07d1f728A80E6e36aE',
 'gasUsed': 61779,
 'logs': [AttributeDict({'address': '0xAD5ce863aE3E4E9394Ab43d4ba0D80f419F61789',
   'blockHash': HexBytes('0x986940a354b39f834c8ee299ce2dbe76612436f0b5e5bd3bfa46e3245cc7a7a8'),
   'blockNumber': 30462235,
   'data': '0x000000000000000000000000000000000000000000000002b5e3af16b1880000',
   'logIndex': 30,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000b09c603ea024b4435d74db07d1f728a80e6e36ae'),
    HexBytes('0x00000000000000000000000016537776395108789fe5cc5420545cab210a7d30')],
   'transactionHash': HexBytes('0x639bf25627f1295e41fd855ab262263a2263365774d9cca426a9f423f086d637'),
   'transa

In [82]:
print('41> '), sleep(20)
set_token_price(link.address, e2w(2.08))
print('42> '), sleep(20)
borrow(bob, 0, e2w(87))
print('43> '), sleep(20)
create_position(cathy, link.address, dai.address)
print('44> '), sleep(20)
send_tx(cathy, link.functions.approve, [pc.address, uint_max], tx_args)
print('45> '), sleep(20)
send_tx(cathy, wbtc.functions.approve, [pc.address, uint_max], tx_args)
print('46> '), sleep(20)
lock(cathy, 0, e2w(20))
print('47> '), sleep(20)
borrow(cathy, 0, e2w(10))
print('48> '), sleep(20)
borrow(cathy, 0, e2w(10))
print('49> '), sleep(20)
borrow(cathy, 0, e2w(10))
print('50> '), sleep(20)
set_token_price(wbtc.address, e2w(10.33))
print('51> '), sleep(20)
lock(bob, 0, e2w(1))
print('52> '), sleep(20)
create_position(alice, link.address, usdt.address) # ATTACK
print('53> '), sleep(20)
set_token_price(wbtc.address, e2w(10.32))
print('54> '), sleep(20)
send_tx(bob, wbtc.functions.approve, [pc.address, uint_max], tx_args)
print('55> '), sleep(20)
lock(bob, 1, 10 ** 8)
print('56> '), sleep(20)
borrow(bob, 1, e2w(13.4))
print('57> '), sleep(20)
borrow(alice, 2, 24 * 10**5) # ATTACK
print('58> '), sleep(20)
borrow(bob, 1, e2w(1.01))
print('59> '), sleep(20)
lock(alice, 0, e2w(10))
print('60> '), sleep(20)
set_token_price(wbtc.address, e2w(9.99))

41> 
42> 
43> 
44> 
45> 
46> 
47> 
48> 
49> 
50> 
51> 
52> 
53> 
54> 
55> 
56> 
57> 
58> 
59> 
60> 


In [85]:
print('61> '), sleep(30)
set_token_price(link.address, e2w(1.997))
print('62> '), sleep(30)
borrow(alice, 2, 20 * 10**6) # ATTACK
print('63> '), sleep(30)
lock(cathy, 0, e2w(3.71))
print('64> '), sleep(30)
set_token_price(link.address, e2w(1.89))
print('65> '), sleep(30)
unlock(cathy, 0, e2w(3.71))
print('66> '), sleep(30)
lock(cathy, 0, e2w(3.71))
print('67> '), sleep(30)
unlock(cathy, 0, e2w(3.71))
print('68> '), sleep(30)
lock(cathy, 0, e2w(3.71))
print('69> '), sleep(30)
unlock(cathy, 0, e2w(3.71))
print('70> '), sleep(30)
set_token_price(link.address, e2w(1.891))
print('71> '), sleep(30)
borrow(alice, 0, e2w(20))
print('72> '), sleep(30)
set_token_price(wbtc.address, e2w(8.99))
print('73> '), sleep(30)
liquidate(alice, bob.address, 1, e2w(14.41))
print('74> '), sleep(30)
set_token_price(wbtc.address, e2w(8.09))
print('75> '), sleep(30)
unlock(bob, 1, 11840935)
print('76> '), sleep(30)
create_position(bob, wbtc.address, dai.address)
print('77> '), sleep(30)
create_position(cathy, wbtc.address, usdt.address) # ATTACK
print('78> '), sleep(30)
set_token_price(wbtc.address, e2w(8))
print('79> '), sleep(30)
create_position(cathy, link.address, usdt.address) # ATTACK
print('80> '), sleep(30)
borrow(alice, 0, e2w(1.1))
print('81> '), sleep(30)
borrow(alice, 1, e2w(1.1))
print('82> '), sleep(30)
repay(alice, 1, e2w(1.1))
print('83> '), sleep(30)
set_token_price(wbtc.address, e2w(7.67))
print('84> '), sleep(30)
repay(alice, 1, e2w(1))
print('85> '), sleep(30)
set_token_price(wbtc.address, e2w(7.62))
print('86> '), sleep(30)
create_position(cathy, link.address, dai.address)
print('87> '), sleep(30)
set_token_price(wbtc.address, e2w(7.1))
print('88> '), sleep(30)
send_tx(cathy, link.functions.approve, [pc.address, uint_max], tx_args)
print('89> '), sleep(30)
lock(cathy, 3, e2w(4.33))
print('90> '), sleep(30)
set_token_price(wbtc.address, e2w(6.5))
print('91> '), sleep(30)
lock(bob, 1, 10**7)
print('92> '), sleep(30)
borrow(cathy, 2, 8 * 10**6) # ATTACK
print('93> '), sleep(30)
borrow(bob, 1, e2w(0.2))
print('94> '), sleep(30)
set_token_price(wbtc.address, e2w(4))
print('95> '), sleep(30)
liquidate(cathy, alice.address, 1, e2w(1.46))
print('96> '), sleep(30)
repay(alice, 1, e2w(0))
print('97> '), sleep(30)
repay(alice, 1, e2w(0.01))
print('98> '), sleep(30)
set_token_price(wbtc.address, e2w(4.2))
print('99> '), sleep(30)
borrow(cathy, 2, int(17.8 * 10**6)) # ATTACK
print('100> '), sleep(30)
borrow(cathy, 3, e2w(0))

61> 
62> 
63> 
64> 
65> 
66> 
67> 
68> 
69> 
70> 
71> 
72> 
73> 
74> 
75> 
76> 
77> 
78> 
79> 
80> 
81> 
82> 
83> 
84> 
85> 
86> 
87> 
88> 
89> 
90> 
91> 
92> 
93> 
94> 
95> 
96> 
97> 
98> 
99> 
100> 


AttributeDict({'blockHash': HexBytes('0x362fad2efa533167d7fe1251a456c11ac546a90b99849f85f8ea9d0fe59fb396'),
 'blockNumber': 30462622,
 'contractAddress': None,
 'cumulativeGasUsed': 170443,
 'effectiveGasPrice': 1500000000,
 'from': '0x941c73fBB405a9024C08A52cBEeAF80D02d2B1A1',
 'gasUsed': 74679,
 'logs': [AttributeDict({'address': '0xFf795577d9AC8bD7D90Ee22b6C1703490b6512FD',
   'blockHash': HexBytes('0x362fad2efa533167d7fe1251a456c11ac546a90b99849f85f8ea9d0fe59fb396'),
   'blockNumber': 30462622,
   'data': '0x0000000000000000000000000000000000000000000000000000000000000000',
   'logIndex': 1,
   'removed': False,
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x00000000000000000000000016537776395108789fe5cc5420545cab210a7d30'),
    HexBytes('0x000000000000000000000000941c73fbb405a9024c08a52cbeeaf80d02d2b1a1')],
   'transactionHash': HexBytes('0x8a52ae6c3f6f3efdf62f183ddd020bdd844d350681238625bf7226e40b884d6d'),
   'transac

In [88]:
print('101> '), sleep(30)
repay(cathy, 3, e2w(0))
print('102> '), sleep(30)
repay(alice, 1, e2w(0.015))
print('103> '), sleep(30)
lock(cathy, 3, e2w(0.007))
print('104> '), sleep(30)
liquidate(cathy, alice.address, 1, e2w(1.46))
print('105> '), sleep(30)
set_token_price(wbtc.address, e2w(4.13))
print('106> '), sleep(30)
borrow(cathy, 1, int(19.9 * 10**6)) # ATTACK
print('107> '), sleep(30)
lock(alice, 1, 7 * 10**7)
print('108> '), sleep(30)
borrow(alice, 1, e2w(1))
print('109> '), sleep(30)
repay(alice, 1, e2w(0.3))
print('110> '), sleep(30)
set_token_price(wbtc.address, e2w(4.888))
print('111> '), sleep(30)
borrow(bob, 1, e2w(0.125))
print('112> '), sleep(30)
borrow(bob, 1, e2w(0.125))
print('113> '), sleep(30)
borrow(bob, 1, e2w(0.125))
print('114> '), sleep(30)
borrow(bob, 1, e2w(0.125))
print('115> '), sleep(30)
set_token_price(wbtc.address, e2w(4.39))
print('116> '), sleep(30)
repay(bob, 1, e2w(0))
print('117> '), sleep(30)
unlock(alice, 1, int(0.35 * 10**8))
print('118> '), sleep(30)
set_token_price(wbtc.address, e2w(4.56691))
print('119> '), sleep(30)
unlock(alice, 1, int(0.15 * 10**8))
print('120> '), sleep(30)
repay(bob, 1, e2w(0))
print('121> '), sleep(30)
set_token_price(link.address, e2w(1.891))
print('122> '), sleep(30)
set_token_price(dai.address, e2w(0.51))
print('123> '), sleep(30)
borrow(cathy, 1, int(2 * 10**6)) # ATTACK
print('124> '), sleep(30)
lock(alice, 0, e2w(20))
print('125> '), sleep(30)
borrow(alice, 0, e2w(2))
print('126> '), sleep(30)
lock(alice, 0, e2w(488.88))
print('127> '), sleep(30)
unlock(alice, 0, e2w(0.88))
print('128> '), sleep(30)
borrow(alice, 0, e2w(100))
print('129> '), sleep(30)
set_token_price(link.address, e2w(2.13))
print('130> '), sleep(30)
borrow(alice, 0, e2w(100))
print('131> '), sleep(30)
borrow(cathy, 1, int(100 * 10**6)) # ATTACK
print('132> '), sleep(30)
repay(alice, 0, e2w(1))
print('133> '), sleep(30)
set_token_price(dai.address, e2w(0.55))
print('134> '), sleep(30)
liquidate(alice, bob.address, 1, e2w(0.7))
print('135> '), sleep(30)
repay(alice, 0, e2w(1.1))
print('136> '), sleep(30)
repay(alice, 1, e2w(0.1))
print('137> '), sleep(30)
lock(bob, 0, e2w(100))
print('138> '), sleep(30)
lock(bob, 0, e2w(100))
print('139> '), sleep(30)
set_token_price(dai.address, e2w(0.57))
print('140> '), sleep(30)
set_token_price(link.address, e2w(2.123))
print('141> '), sleep(30)
borrow(bob, 0, e2w(0.125))
print('142> '), sleep(30)
borrow(bob, 0, e2w(125))
print('143> '), sleep(30)
set_token_price(link.address, e2w(2.009))
print('144> '), sleep(30)
create_position(bob, link.address, dai.address)
print('145> '), sleep(30)
lock(bob, 3, e2w(88))
print('146> '), sleep(30)
unlock(alice, 0, e2w(88))
print('147> '), sleep(30)
lock(bob, 3, e2w(88))
print('148> '), sleep(30)
set_token_price(link.address, e2w(2))
print('149> '), sleep(30)
repay(alice, 1, e2w(0.39))
print('150> '), sleep(30)
unlock(alice, 1, 10**7)
print('151> '), sleep(30)
set_token_price(dai.address, e2w(0.58))
print('152> '), sleep(30)
borrow(cathy, 1, int(88 * 10**6)) # ATTACK
print('153> '), sleep(30)
borrow(cathy, 2, int(1 * 10**6)) # ATTACK
print('154> '), sleep(30)
set_token_price(dai.address, e2w(0.59))
print('155> '), sleep(30)
borrow(alice, 2, int(10.7 * 10**6)) # ATTACK
print('156> '), sleep(30)
unlock(alice, 0, e2w(0.88))
print('157> '), sleep(30)
set_token_price(wbtc.address, e2w(4.69))
print('158> '), sleep(30)
set_token_price(dai.address, e2w(0.62))
print('159> '), sleep(30)
borrow(bob, 3, e2w(66))
print('160> '), sleep(30)
borrow(cathy, 0, e2w(6))
print('161> '), sleep(30)
borrow(cathy, 0, e2w(1.2))
print('162> '), sleep(30)
set_token_price(wbtc.address, e2w(4.9999))
print('163> '), sleep(30)
set_token_price(dai.address, e2w(0.69))
print('164> '), sleep(30)
repay(bob, 0, e2w(44))
print('165> '), sleep(30)
set_token_price(link.address, e2w(1.5))
print('166> '), sleep(30)
lock(bob, 3, e2w(239))
print('167> '), sleep(30)
borrow(alice, 0, e2w(169.9))
print('168> '), sleep(30)
borrow(bob, 3, e2w(111))
print('169> '), sleep(30)
set_token_price(link.address, e2w(1.1))
print('170> '), sleep(30)
borrow(cathy, 3, e2w(2.2))
print('171> '), sleep(30)
repay(alice, 0, e2w(39))
print('172> '), sleep(30)
repay(bob, 3, e2w(33))
print('173> '), sleep(30)
set_token_price(link.address, e2w(0.98))
print('174> '), sleep(30)
set_token_price(dai.address, e2w(0.7))
print('175> '), sleep(30)
borrow(alice, 2, int(49 * 10**6)) # ATTACK
print('176> '), sleep(30)
repay(bob, 3, e2w(3))
print('177> '), sleep(30)
borrow(cathy, 3, e2w(1.01))
print('178> '), sleep(30)
set_token_price(link.address, e2w(1.5))
print('179> '), sleep(30)
borrow(alice, 0, e2w(1.2))
print('180> '), sleep(30)
borrow(bob, 0, e2w(1.2))
print('181> '), sleep(30)
set_token_price(link.address, e2w(1.6))
print('182> '), sleep(30)
set_token_price(dai.address, e2w(0.71))
print('183> '), sleep(30)
set_token_price(wbtc.address, e2w(5.7))
print('184> '), sleep(30)
create_position(bob, wbtc.address, dai.address)
print('185> '), sleep(30)
borrow(bob, 0, e2w(3))
print('186> '), sleep(30)
lock(bob, 4, int(0.01 * 10**8))
print('187> '), sleep(30)
borrow(bob, 4, e2w(0.008))
print('188> '), sleep(30)
repay(bob, 4, e2w(0.008))
print('189> '), sleep(30)
borrow(cathy, 2, int(3 * 10**6)) # ATTACK
print('190> '), sleep(30)
create_position(alice, link.address, usdt.address) # ATTACK
print('191> '), sleep(30)
borrow(alice, 1, e2w(0.3))
print('192> '), sleep(30)
borrow(alice, 2, int(0.3 * 10**6)) # ATTACK
print('193> '), sleep(30)
borrow(alice, 3, int(3 * 10**6)) # ATTACK
print('194> '), sleep(30)
create_position(alice, wbtc.address, dai.address)
print('195> '), sleep(30)
set_token_price(link.address, e2w(2.4))
print('196> '), sleep(30)
borrow(cathy, 1, 27 * 10**6) # ATTACK
print('197> '), sleep(30)
repay(cathy, 0, e2w(1.1))
print('198> '), sleep(30)
borrow(bob, 4, e2w(0.001))
print('199> '), sleep(30)
borrow(bob, 4, e2w(0.0015))
print('200> '), sleep(30)
borrow(bob, 0, e2w(2.1))
print('201> '), sleep(30)
unlock(bob, 1, 726772)
print('202> '), sleep(30)
create_position(bob, wbtc.address, dai.address)
print('203> '), sleep(30)
create_position(bob, wbtc.address, usdt.address) # ATTACK
print('204> '), sleep(30)
borrow(cathy, 3, e2w(2))
print('205> '), sleep(30)
set_token_price(link.address, e2w(3))
print('206> '), sleep(30)
lock(bob, 5, 1567707)
print('207> '), sleep(30)
borrow(bob, 5, e2w(0.05))
print('208> '), sleep(30)
repay(alice, 1, e2w(0.3))
print('209> '), sleep(30)
repay(bob, 5, e2w(0.01))
print('210> '), sleep(30)
borrow(bob, 6, 11 * 10**6) # ATTACK
print('211> '), sleep(30)
borrow(bob, 5, e2w(0.01))
print('212> '), sleep(30)
set_token_price(link.address, e2w(2.2))
print('213> '), sleep(30)
set_token_price(dai.address, e2w(0.78))
print('214> '), sleep(30)
borrow(bob, 5, e2w(0.02))
print('215> '), sleep(30)
repay(bob, 3, e2w(4))
print('216> '), sleep(30)
borrow(bob, 5, e2w(0.02))
print('217> '), sleep(30)
set_token_price(dai.address, e2w(0.88))
print('218> '), sleep(30)
liquidate(cathy, bob.address, 5, e2w(0.01))
print('219> '), sleep(30)
set_token_price(dai.address, e2w(0.98))
print('220> '), sleep(30)
liquidate(cathy, bob.address, 5, e2w(0.01))
print('221> '), sleep(30)
liquidate(cathy, bob.address, 5, e2w(0.01))
print('222> '), sleep(30)
borrow(cathy, 2, 14 * 10**6) # ATTACK
print('223> '), sleep(30)
set_token_price(dai.address, e2w(1))
print('224> '), sleep(30)
repay(bob, 5, e2w(0.06))
print('225> '), sleep(30)
unlock(bob, 5, 1019639)
print('226> '), sleep(30)
create_position(bob, wbtc.address, dai.address)
print('227> '), sleep(30)
repay(alice, 1, e2w(0.19))
print('228> '), sleep(30)
repay(cathy, 3, e2w(1))
print('229> '), sleep(30)
unlock(cathy, 3, e2w(1))
print('230> '), sleep(30)
send_tx(owner, pc.functions.pause, [], tx_args) # END

101> 
102> 
103> 
104> 
105> 
106> 
107> 
108> 
109> 
110> 
111> 
112> 
113> 
114> 
115> 
116> 
117> 
118> 
119> 
120> 
121> 
122> 
123> 
124> 
125> 
126> 
127> 
128> 
129> 
130> 
131> 
132> 
133> 
134> 
135> 
136> 
137> 
138> 
139> 
140> 
141> 
142> 
143> 
144> 
145> 
146> 
147> 
148> 
149> 
150> 
151> 
152> 
153> 
154> 
155> 
156> 
157> 
158> 
159> 
160> 
161> 
162> 
163> 
164> 
165> 
166> 
167> 
168> 
169> 
170> 
171> 
172> 
173> 
174> 
175> 
176> 
177> 
178> 
179> 
180> 
181> 
182> 
183> 
184> 
185> 
186> 
187> 
188> 
189> 
190> 
191> 
192> 
193> 
194> 
195> 
196> 
197> 
198> 
199> 
200> 
201> 
202> 
203> 
204> 
205> 
206> 
207> 
208> 
209> 
210> 
211> 
212> 
213> 
214> 
215> 
216> 
217> 
218> 
219> 
220> 
221> 
222> 
223> 
224> 
225> 
226> 
227> 
228> 
229> 
230> 


AttributeDict({'blockHash': HexBytes('0xc0afd56b714cb2a35c2460c0ea68e9918457cd4d1baa1b1b66e6f00e78bec7aa'),
 'blockNumber': 30463563,
 'contractAddress': None,
 'cumulativeGasUsed': 131607,
 'effectiveGasPrice': 1500000000,
 'from': '0x85BFba62f1b0bb5ad63Dc15611B96A3457095390',
 'gasUsed': 28073,
 'logs': [AttributeDict({'address': '0x16537776395108789FE5cC5420545CAb210a7D30',
   'blockHash': HexBytes('0xc0afd56b714cb2a35c2460c0ea68e9918457cd4d1baa1b1b66e6f00e78bec7aa'),
   'blockNumber': 30463563,
   'data': '0x00000000000000000000000085bfba62f1b0bb5ad63dc15611b96a3457095390',
   'logIndex': 2,
   'removed': False,
   'topics': [HexBytes('0x62e78cea01bee320cd4e420270b5ea74000d11b0c9f74754ebdbfc544b05a258')],
   'transactionHash': HexBytes('0x0578e2f1a4157611bd524d63d65badf631c2b2b3344a2883c7e2f2cd43acec77'),
   'transactionIndex': 2,
   'transactionLogIndex': '0x0',
   'type': 'mined'})],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000

# save

In [11]:
addr2name = {}
addr2name[zero_addr] = 'eth'
addr2name[alice.address] = 'alice'
addr2name[bob.address] = 'bob'
addr2name[cathy.address] = 'cathy'
addr2name[weth.address] = 'weth'
addr2name[wbtc.address] = 'wbtc'
addr2name[link.address] = 'link'
addr2name[dai.address] = 'dai'
addr2name[usdt.address] = 'usdt'

name2addr = {}
name2addr['alice'] = alice.address
name2addr['bob'] = bob.address
name2addr['cathy'] = cathy.address

name2obj = {}
name2obj['alice'] = alice
name2obj['bob'] = bob
name2obj['cathy'] = cathy
name2obj['weth'] = weth
name2obj['wbtc'] = wbtc
name2obj['link'] = link
name2obj['dai'] = dai
name2obj['usdt'] = usdt
# name2obj['alice'] = alice

In [12]:
def print_balances(acct):
    print('-'*10)
    print('address:', addr2name[acct.address])
    print('eth:', w2e(Bal(0, acct.address)))
    print('weth:', w2e(Bal(weth, acct.address)))
    print('wbtc:', Bal(wbtc, acct.address) / (10**8))
    print('link:', w2e(Bal(link, acct.address)))
    print('dai:', w2e(Bal(dai, acct.address)))
    print('usdt:', Bal(usdt, acct.address) / (10**6))
    
def scale_balance(name, value):
    if name in ['eth', 'weth', 'link', 'dai']:
        return w2e(value)
    elif name in ['wbtc']:
        return value / (10**8)
    elif name in ['usdt']:
        return value / (10**6)

def print_position(acct, pid, l):
    print(f'===== {pid} =====')
    print('lock_token:', addr2name[l[1]])
    print('locked:', scale_balance(addr2name[l[1]], l[0]))
    print('borrow_token:', addr2name[l[3]])
    print('borrowed:', scale_balance(addr2name[l[3]], l[2]))
    print('health_factor:', get_health_factor(acct, pid))

def print_prices():
    print('wbtc price:', w2e(oracle.functions.getTokenPrice(wbtc.address).call()[0]))
    print('link price:', w2e(oracle.functions.getTokenPrice(link.address).call()[0]))
    print('dai price:', w2e(oracle.functions.getTokenPrice(dai.address).call()[0]))
    print('usdt price:', w2e(oracle.functions.getTokenPrice(usdt.address).call()[0]))

In [13]:
def print_all():
    print_prices()
    for acct in [alice, bob, cathy]:
        print_balances(acct)

        for pid in range(addr2pid[acct.address]):
            print_position(acct, pid, get_position(acct, pid))

In [563]:
addr2pid = {}
addr2pid[alice.address] = 5
addr2pid[bob.address] = 8
addr2pid[cathy.address] = 4

In [559]:
print_all()

wbtc price: 5.7
link price: 2.2
dai price: 1
usdt price: 0
----------
address: alice
eth: 0.004870046843747602
weth: 0
wbtc: 1.47432293
link: 570.87
dai: 332.076
usdt: 85.4
===== 0 =====
lock_token: link
locked: 429.13
borrow_token: dai
borrowed: 343.021
health_factor: 2.201815049224391509
===== 1 =====
lock_token: wbtc
locked: 0.1
borrow_token: dai
borrowed: 0.21
health_factor: 2.171428571428571428
===== 2 =====
lock_token: link
locked: 0
borrow_token: usdt
borrowed: 82.4
health_factor: 1
===== 3 =====
lock_token: link
locked: 0
borrow_token: usdt
borrowed: 3.0
health_factor: 1
===== 4 =====
lock_token: wbtc
locked: 0.0
borrow_token: dai
borrowed: 0
health_factor: 1
----------
address: bob
eth: 0.004349675629169454
weth: 0
wbtc: 0.0
link: 300.698
dai: 10447.5675
usdt: 10011.0
===== 0 =====
lock_token: link
locked: 276.66222694255631081
borrow_token: dai
borrowed: 231.245
health_factor: 2.105669395744336556
===== 1 =====
lock_token: wbtc
locked: 0.0
borrow_token: dai
borrowed: 0
health

In [172]:
l = [
    #(0, encode_abi(['address', 'address'], [zero_addr, usdc.address])),
    (1, encode_abi(['uint', 'uint'], [6, e2w(1)])),
    (1, encode_abi(['uint', 'uint'], [6, e2w(1)])),
    #(3, encode_abi(['uint', 'uint'], [6, e2w(1)])),
    (2, encode_abi(['uint', 'uint'], [6, e2w(2)])),
]
actions = [x[0] for x in l]
data = [x[1] for x in l]
# print(actions, data)
pc.functions.batch(actions, data).transact({'from': alice, 'value': e2w(1)})

HexBytes('0xa9cb80dae0b0db16b37ed3bb1a0fa0021927cc41df78432f63fc6dea90f1017e')